In [14]:
import utils

utils.load_env()

from langchain.llms import OpenAI

# Create an instance of the OpenAI class
openai = OpenAI(model_name="text-davinci-003")

# Get the maximum context size for the model
max_context_size = openai.modelname_to_contextsize(openai.model_name)

# Set the max_tokens attribute to a value within the model's maximum context length
openai.max_tokens = max_context_size - 1  # or any value less than max_context_size

# Alternatively, you can use the max_tokens_for_prompt method
prompt = "Tell me a joke."*800
openai.max_tokens = openai.max_tokens_for_prompt(prompt)
openai.max_tokens

97